In [1]:
import geopandas as gpd
import rasterio
import tools
import numpy as np

### SOIL MAP (FROM ANNELI)

In [2]:
file1= r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J1.shp'
file2 = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J2.shp'

df1 = gpd.read_file(file1)
df2 = gpd.read_file(file2)
codes = {}
for id in df1['JY1'].unique():
    codes[id] = df1.loc[df1['JY1'] == id, 'JY1_TX'].iloc[0]
for id in df2['JG2'].unique():
    codes[id] = df2.loc[df2['JG2'] == id, 'JG2_TX'].iloc[0]
print(codes)

{np.int64(100): 'Morän', np.int64(84): 'Postglacial sand--grus', np.int64(89): 'Svallsediment, grus--block', np.int64(75): 'Torv', np.int64(50): 'Isälvssediment', np.int64(86): 'Lera--silt', np.int64(200): 'Fyllning', np.int64(31): 'Postglacial sand', np.int64(8809): 'Älvsediment, sand', np.int64(91): 'Vatten', np.int64(888): 'Berg'}


In [3]:
resos = [20, 40, 80, 160]
id = 0
for reso in resos:
    id += 1
    # Reference file path
    ref_fp = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_{reso}.asc'
    out_fp1 = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J1_{reso}.asc'
    out_fp2 = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J2_{reso}.asc'
    out_fp3 = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J1_J2_{reso}.asc'
    out_fp4 = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/lake_mask_{reso}.asc'
    out_fp5 = f'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SOIL/krycklan_QD/krycklan_QD_J1_J2_final_{reso}.asc'

    tools.rasterize_shapefile(shapefile=file1, burn_field='JY1', subset=None, out_fp=out_fp1, ref_raster=ref_fp, plot=False, save_in='asc')
    tools.rasterize_shapefile(shapefile=file2, burn_field='JG2', subset=None, out_fp=out_fp2, ref_raster=ref_fp, plot=False, save_in='asc')
    tools.fill_layer_na_with_layer(priority_layer=out_fp1, secondary_layer=out_fp2, out_fp=out_fp3, save_in='asc')
    
    with rasterio.open(out_fp3) as src:
        data = src.read(1)
        meta = src.meta.copy()
        new_data = data.copy()
        new_data[data != 91.0] = meta['nodata']
        new_data[data == 91.0] = 1.0
    with rasterio.open(out_fp4, 'w', **meta) as dst:
        out = dst.write(new_data, 1)

    with rasterio.open(out_fp3) as src:
        data = src.read(1)
        new_data = data.copy()
        meta = src.meta.copy()
        nodata = meta['nodata']
        if id == 1:
            orig_uniq = np.unique(data[data != nodata])
            new_uniq = np.arange(1, len(orig_uniq)+1)
        for i in range(len(orig_uniq)):
            new_data[data == orig_uniq[i]] = new_uniq[i] 
    with rasterio.open(out_fp5, 'w', **meta) as dst:
        out = dst.write(new_data, 1)